In [23]:
import numpy as np
import pandas as pd
from faker import Faker

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

from datetime import datetime

In [24]:
fake = Faker('it_IT')

In [25]:
def timestamp(dt):
    epoch = datetime.utcfromtimestamp(0)
    return (dt - epoch.date()).total_seconds() * 1000.0

Si creano i vettori da usare per generare dati

In [26]:
# GENERATORS
SESSO = ["Donna", "Uomo", "Altro"]
ANNO = ['2020', '2021']
MESE = ["Gen", "Feb", "Mar", "Apr", "Mag", "Giu", "Lug", "Ago", "Set", "Ott", "Nov", "Dic"]
REGIONE = ["Sicilia", "Piemonte", "Marche", "Valle d'Aosta", "Toscana", "Campania", "Puglia", "Veneto",
           "Lombardia", "Emilia-Romagna", "Trentino-Alto Adige", "Sardegna", "Molise", "Calabria",
           "Abruzzo", "Lazio", "Liguria", "Friuli-Venezia Giulia", "Basilicata", "Umbria"]
PELLE = ["Molto chiara", "Chiara", "Intermedia", "Scura", "Molto scura"]
ISTRUZIONE = ["Primaria", "Secondaria", "Laurea Breve", "Master/PhD"]
ATTIVITA = ["Pesante", "Moderata", "Leggera"]
ABITUDINE = ["Nessuna", "Lieve", "Ex", "Moderata", "Intensa"]
DIETA = ["Poco equilibrata", "Regolare", "Molto equilibrata"]
TIMING = ["Meno di 1 anno fa", "1-3 anni fa", "3-5 anni fa", "Oltre 5 anni fa"]
SINTOMI = ["Lievi", "Moderati", "Intensi"]
VISITE = [
    "Agoaspirato", "Marcatori tumorali", "Colonscopia",
    "Visita odontoiatrica",
    "Ecografia mammaria", "Ecografia transvaginale", "Hpv Dna Test", "Pap-Test", "Mammografia",
    "Elettrocardiogramma", "Holter", "Coronarografia", "Ecocardiografia",
    "Biopsia prostatica",
    "Spirometria", "Esame del sangue",
    "Pet", "Risonanza magnetica (RM)", "Scintigrafia",
]
AREE = ["Oncologica", "Odontoiatrica", "Ginecologica",
        "Cardiologica", "Andrologica", "Generale", "Diagnostica"]

In [27]:
# QUESTIONARI
COLUMNS_USER = [
    "data",
    "nome_cognome",
    "data_iscrizione",
    "eta",
    "sesso",
    "regione",
    "peso",
    "altezza",
    "pelle",
    "istruzione",
    "quiz_categorieRischio",
    "quiz_covid",
    "quiz_familiarita",
    "quiz_salutePersonale",
    "quiz_stileDiVita",
    "quiz_vaccini",
    "diabete",
    "diag_diabete",
    "cardiopatia",
    "ipertensione",
    "attivita_lavoro",
    "attivita_tl",
    "fumo",
    "alcol",
    "sostanze",
    "dieta"
    "fam_onco",
    "fam_meta",
    "fam_cardio",
    "fam_resp",
    "fam_onco_gino",
    "fam_onco_andro"
    "vacc_pnmccc",
    "vacc_zoster",
    "vacc_influ"
    "cup_derma",
    "cup_sangue",
    "cup_odonto",
    "cup_glicemia",
    "cup_cole",
    "cup_cardio",
    "cup_oculo",
    "cup_spiro",
    "cup_feci",
    "cup_andro",
    "cup_colon",
    "cup_gino",
    "cup_seno",
    "cup_tac",
    "cup_onco"
    "covid",
    "sintomi",
    "vaccino"
]
COLUMNS_VISIT = [
    "Data",
    "Eseguita",
    "Nome",
    "Tipologia",
    "Prenotata",
]

In [28]:
def create_users():
    my_row = {}
    my_visit = {}
    # IDENTIFICAZIONE
    my_row['data'] = fake.date_between_dates(date_start=datetime(2020, 1, 1))
    my_row['data_iscrizione'] = timestamp(dt=my_row['data'])
    my_row['sesso'] = np.random.choice(SESSO, 1)[0]
    if my_row['sesso'] == 'Donna':
        my_row['nome_cognome'] = fake.name_female()
    elif my_row['sesso'] == 'Uomo':
        my_row['nome_cognome'] = fake.name_male()
    else:
        my_row['nome_cognome'] = fake.name()
    my_row['mail'] = my_row['nome_cognome'].lower().replace(' ', '.') + '@gmail.com'
    my_row['eta'] = fake.pyint(min_value=18, max_value=90, step=1)
    my_row['regione'] = np.random.choice(REGIONE, 1)[0]
    my_row['peso'] = fake.pyint(min_value=40, max_value=140, step=1)
    if my_row['peso'] > 90:
        my_row['altezza'] = fake.pyint(min_value=160, max_value=205, step=1)
    else:
        my_row['altezza'] = fake.pyint(min_value=140, max_value=180, step=1)
    my_row['pelle'] = np.random.choice(PELLE, 1)[0]
    if my_row['eta'] < 26:
        my_row['istruzione'] = np.random.choice(ISTRUZIONE[:-1], 1)[0]
    else:
        my_row['istruzione'] = np.random.choice(ISTRUZIONE, 1)[0]
    
    # CATEGORIA A RISCHIO
    my_row['diabete'] = fake.pybool()
    if my_row['diabete']:
        my_row['diag_diabete'] = fake.date().split('-')[0]
    my_row['cardiopatia'] = fake.pybool()
    my_row['ipertensione'] = fake.pybool()
    
    # STILE  DI VITA
    my_row['attivita_lavoro'] = np.random.choice(ATTIVITA, 1)[0]
    my_row['attivita_tl'] = np.random.choice(ATTIVITA, 1)[0]
    my_row['fumo'] = np.random.choice(ABITUDINE, 1)[0]
    my_row['alcol'] = np.random.choice(ABITUDINE, 1)[0]
    my_row['sostanze'] = np.random.choice(ABITUDINE, 1)[0]
    my_row['dieta'] = np.random.choice(DIETA, 1)[0]
    
    # FAMILIARITA
    my_row['fam_onco'] = fake.pybool()
    my_row['fam_meta'] = fake.pybool()
    my_row['fam_cardio'] = fake.pybool()
    my_row['fam_resp'] = fake.pybool()
    if my_row['sesso'] == 'Donna' or my_row['sesso'] == 'Altro':
        my_row['fam_onco_gino'] = fake.pybool()
    if my_row['sesso'] == 'Uomo' or my_row['sesso'] == 'Altro':
        my_row['fam_onco_andro'] = fake.pybool()
    
    # VACCINI
    if my_row['eta'] >= 65:
        my_row['vacc_pnmccc'] = fake.pybool()
        my_row['vacc_zoster'] = fake.pybool()
        my_row['vacc_influ'] = fake.pybool()
    
    # ULTIMO CHECK-UP
    if my_row['pelle'] in PELLE[0:3]:
        my_row['cup_derma'] = np.random.choice(TIMING, 1)[0]
    my_row['cup_sangue'] = np.random.choice(TIMING, 1)[0]
    my_row['cup_odonto'] = np.random.choice(TIMING, 1)[0]
    if my_row['eta'] >= 25:
        my_row['cup_cardio'] = np.random.choice(TIMING, 1)[0]
        my_row['cup_oculo'] = np.random.choice(TIMING, 1)[0]
        if my_row['dieta'] == DIETA[0] or my_row['fam_meta']:
            my_row['cup_glicemia'] = np.random.choice(TIMING, 1)[0]
            if my_row['attivita_tl'] == ATTIVITA[2]:
                my_row['cup_cole'] = np.random.choice(TIMING, 1)[0]
    if my_row['sesso'] == 'Donna' or my_row['sesso'] == 'Altro':
        my_row['cup_gino'] = np.random.choice(TIMING, 1)[0]
        if my_row['eta'] >= 30:
            my_row['cup_seno'] = np.random.choice(TIMING, 1)[0]
    if my_row['sesso'] == 'Uomo' or my_row['sesso'] == 'Altro':
        my_row['cup_andro'] = np.random.choice(TIMING, 1)[0]
    if my_row['eta'] >= 40:
        my_row['cup_feci'] = np.random.choice(TIMING, 1)[0]
    if my_row['eta'] >= 50:
        my_row['cup_colon'] = np.random.choice(TIMING, 1)[0]
    if my_row['fam_onco']:
        my_row['cup_onco'] = np.random.choice(TIMING, 1)[0]
        if my_row['alcol'] in ABITUDINE[2:] or my_row['fumo'] in ABITUDINE[2:]:
            my_row['cup_tac'] = np.random.choice(TIMING, 1)[0]
    
    if bool(set(my_row.keys()) & set(RISCHIO)):
        my_row['quiz_categorieRischio'] = True
    if bool(set(my_row.keys()) & set(FAMILIARITA)):
        my_row['quiz_familiarita'] = True
    if bool(set(my_row.keys()) & set(COVID)):
        my_row['quiz_covid'] = True
    if bool(set(my_row.keys()) & set(VACCINI)):
        my_row['quiz_vaccini'] = True
    if bool(set(my_row.keys()) & set(STILE)):
        my_row['quiz_stileDiVita'] = True
    if bool(set(my_row.keys()) & set(CUP)):
        my_row['quiz_salutePersonale'] = True
    
    my_visit['mail'] = my_row['mail']
    my_visit['Data'] = timestamp(dt=fake.date_between_dates(date_start=my_row['data']))
    my_visit['Nome'] = np.random.choice(VISITE, 1)[0]
    if my_visit['Nome'] in VISITE[0:3]:
        my_visit['Tipologia'] = "Oncologica"
    elif my_visit['Nome'] == 'Visita odontoiatrica':
        my_visit['Tipologia'] = "Odontoiatrica"
    elif my_visit['Nome'] in VISITE[4:9]:
        my_visit['Tipologia'] = "Ginecologica"
    elif my_visit['Nome'] in VISITE[9:13]:
        my_visit['Tipologia'] = "Cardiologica"
    elif my_visit['Nome'] == 'Biopsia prostatica':
        my_visit['Tipologia'] = "Andrologica"
    elif my_visit['Nome'] in VISITE[14:16]:
        my_visit['Tipologia'] = "Generale"
    elif my_visit['Nome'] in VISITE[16:]:
        my_visit['Tipologia'] = "Diagnostica"
    my_visit['Prenotata'] = fake.pybool()
    my_visit['Eseguita'] = fake.pybool()
    
    gen_u = (k for k in COLUMNS_USER if k not in my_row.keys())
    for k in gen_u:
        my_row[k] = None
    
    gen_v = (k for k in COLUMNS_VISIT if k not in my_visit.keys())
    for k in gen_v:
        my_visit[k] = None
    
    return my_row, my_visit

In [31]:
output_users = []
output_visits = []
for i in range(20):
    my_user_row, my_visit_row = create_users()
    output_users.append(my_user_row)
    output_visits.append(my_visit_row)

In [32]:
output_users_df = pd.DataFrame(output_users)
output_visits_df = pd.DataFrame(output_visits)

In [33]:
output_users_df

,data,data_iscrizione,sesso,nome_cognome,mail,eta,regione,peso,altezza,pelle,...,cup_spiro,cup_andro,cup_colon,cup_oncocovid,sintomi,vaccino,fam_onco_andro,vacc_pnmccc,vacc_influ,cup_derma
0,2020-04-06,1.586131e+12,Donna,Marisa Bragadin,marisa.bragadin@gmail.com,41,Umbria,110,187,Scura,...,None,None,None,None,None,None,NaN,NaN,NaN,NaN
1,2021-03-16,1.615853e+12,Altro,Sig. Achille Polizzi,sig..achille.polizzi@gmail.com,39,Lazio,114,197,Scura,...,None,3-5 anni fa,None,None,None,None,False,NaN,NaN,NaN
2,2020-08-05,1.596586e+12,Altro,Sig.ra Flavia Argento,sig.ra.flavia.argento@gmail.com,68,Calabria,62,178,Molto chiara,...,None,Meno di 1 anno fa,Meno di 1 anno fa,None,None,None,False,False,True,3-5 anni fa
3,2020-11-28,1.606522e+12,Altro,Ignazio Dellucci-Platini,ignazio.dellucci-platini@gmail.com,67,Veneto,56,150,Molto scura,...,None,3-5 anni fa,1-3 anni fa,None,None,None,False,True,True,NaN
4,2020-03-06,1.583453e+12,Donna,Loretta Versace,loretta.versace@gmail.com,65,Campania,133,161,Scura,...,None,None,1-3 anni fa,None,None,None,NaN,False,False,NaN
5,2020-08-19,1.597795e+12,Uomo,Dott. Pierpaolo Venier,dott..pierpaolo.venier@gmail.com,52,Lombardia,61,169,Intermedia,...,None,Oltre 5 anni fa,Oltre 5 anni fa,None,None,None,True,NaN,NaN,Oltre 5 anni fa
6,2020-02-01,1.580515e+12,Altro,Temistocle Maderna,temistocle.maderna@gmail.com,49,Lombardia,77,151,Scura,...,None,1-3 anni fa,None,None,None,None,True,NaN,NaN,NaN
7,2020-09-27,1.601165e+12,Altro,Adelmo Caboto,adelmo.caboto@gmail.com,60,Sicilia,74,176,Molto chiara,...,None,Meno di 1 anno fa,Meno di 1 anno fa,None,None,None,True,NaN,NaN,3-5 anni fa
8,2020-10-26,1.603670e+12,Altro,Ornella Mondaini,ornella.mondaini@gmail.com,51,Friuli-Venezia Giulia,89,169,Molto scura,...,None,Meno di 1 anno fa,Meno di 1 anno fa,None,None,None,True,NaN,NaN,NaN
9,2020-10-03,1.601683e+12,Donna,Antonietta Magnani,antonietta.magnani@gmail.com,70,Umbria,118,201,Molto scura,...,None,None,1-3 anni fa,None,None,None,NaN,True,True,NaN


In [34]:
output_visits_df

,mail,Data,Nome,Tipologia,Prenotata,Eseguita
0,marisa.bragadin@gmail.com,1.610150e+12,Biopsia prostatica,Andrologica,True,False
1,sig..achille.polizzi@gmail.com,1.617581e+12,Coronarografia,Cardiologica,True,False
2,sig.ra.flavia.argento@gmail.com,1.612483e+12,Ecografia mammaria,Ginecologica,True,True
3,ignazio.dellucci-platini@gmail.com,1.612742e+12,Pet,Diagnostica,False,True
4,loretta.versace@gmail.com,1.612829e+12,Ecografia transvaginale,Ginecologica,True,True
5,dott..pierpaolo.venier@gmail.com,1.612656e+12,Scintigrafia,Diagnostica,True,False
6,temistocle.maderna@gmail.com,1.588464e+12,Hpv Dna Test,Ginecologica,True,False
7,adelmo.caboto@gmail.com,1.614470e+12,Agoaspirato,Oncologica,True,False
8,ornella.mondaini@gmail.com,1.610410e+12,Biopsia prostatica,Andrologica,False,False
9,antonietta.magnani@gmail.com,1.611187e+12,Visita odontoiatrica,Odontoiatrica,False,False


In [36]:
output_users_df.to_csv(f'assets/users_{datetime.now().strftime("%Y-%m-%d_%H%M%S")}.csv')
output_visits_df.to_csv(f'assets/visits_{datetime.now().strftime("%Y-%m-%d_%H%M%S")}.csv')